In [2]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [ ]:
#como el navegador a usar es chrome, hemos descargado ya el driver de la página, guardamos la ruta en una variable
#chrome_path = 'chromedriver_linux64/chromedriver'
#driver = webdriver.Chrome(executable_path = chrome_path)

In [2]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/78.0.3904.108 Safari/537.36'}

r = requests.get(url)
soup = BeautifulSoup(r.content, 'lxml') #usamos lxml como parser

¿Qué queremos obtener de la página?
1. Nombre del producto
2. Precio (?)
3. Detalles
4. Ingredientes

## 1 OBTENER TODAS LAS URLS DE LOS PRODUCTOS

Para ello, utilizaremos bucles que iteren sobre los elementos de la página.
* Variable global cleansers_url, es el contenedor de todas las urls.
* Los links están dentro de un tag 'a', como sólo nos interesan aquellas que tengan href, pues href=True. 
* Se ha agregado class='product-image" porque si no, el bucle cogía todas las urls (wishlist, url de la imagen etc), y solo nos interesan los productos.
* Append cada uno de los links al contenedor.

In [4]:
cleansers_url = []
for i in range (1, 92):
    url = f'https://www.stylevana.com/en_EU/skincare/facial-cleanser/face-wash-cleansers.html?p={i}' 
    r = requests.get(url)
    soup = BeautifulSoup(r.content, 'lxml')
    cleansers_list = soup.find_all('div', class_='product-item-info')
    for item in cleansers_list:
        for link in item.find_all('a', class_='product-image', href=True): 
            cleansers_url.append(link['href'])
print(len(cleansers_url))

1087


with open("cleansers_url.txt", "w") as output:
    output.write(str(cleansers_url))

In [14]:
cleansers_url[12]

'https://www.stylevana.com/en_EU/farm-stay-76-green-tea-seed-premium-moisture-foam-cleansing.html'

## 2 OBTENER LA INFO DE LA PÁGINA PROPIA DEL PRODUCTO
Info que necesitamos:
* Nombre del producto
* Details
* Ingredients

* Details e Ingredients se encuentran dentro de un grupo <div class='panel-group'

In [11]:
test_url = 'https://www.stylevana.com/en_EU/iunik-calendula-complete-cleansing-oil-1.html'
r = requests.get(test_url, headers=headers) #para cada link en el listado, hacemos una requests
soup = BeautifulSoup(r.content, 'lxml')
price = (soup.find('span', class_='price').text.strip())
best_seller = soup.find(class_="product-badge-icon").get('title')

In [12]:
print(price)
print(best_seller)

15.19€
Best Sellers


In [17]:
cleansers = []

In [20]:
for link in cleansers_url[243:1087]:
    r = requests.get(link, headers=headers) #para cada link en el listado, hacemos una requests
    soup = BeautifulSoup(r.content, 'lxml')
    name = soup.find('h1', class_='product-name-h1').text
    #usamos try, except porque hay ocasiones en las que no hay info sobre los ingredientes
    try:
        details = (soup.find('div', id='product-view-details').text.strip()) # details
        price = (soup.find('span', class_='price').text.strip())
        best_seller = soup.find(class_="product-badge-icon").get('title')
        ingredients = soup.find_all('div', id="product-view-ingredients") #ingredients
        for item in ingredients:
            ingredients = item.find('div', class_='panel-body').text.strip()
    except:
        details = 'no info'
        ingredients = 'no info'
        price = 'no info'
        best_seller = 'no'
        
    cleansers_info = {
    'name': name,
    'details': details,
    'ingredients': ingredients,
    'price': price,
    'best-seller': best_seller}
    
    cleansers.append(cleansers_info)

In [21]:
print(len(cleansers))

1087


## 3. PASARLO A DATAFRAME

In [26]:
cleansers_df = pd.DataFrame(cleansers)
cleansers_df

,name,details,ingredients,price,best-seller
0,iUNIK - Calendula Complete Cleansing Oil - 200ml,Product Type: Cleansing Oil\n\n\nSkin Concern:...,[],15.19€,Best Sellers
1,Krave - Matcha Hemp Hydrating Cleanser 120ml,Product Type: Cleansing Gel\n\n\nSkin Concern:...,[],31.29€,Best Sellers
2,heimish - All Clean Balm,Cruelty Free: Yes\n\n\nProduct Type: Cleansing...,"Ethylhexyl Palmitate, Cetyl Ethylhexanoate, PE...",14.89€,Best Sellers
3,COSRX - Low pH Good Morning Gel Cleanser - 150ml,Cruelty Free: Yes\n\n\nProduct Type: Cleansing...,"Water, Cocamidopropyl Betaine, Sodium Lauroyl ...",9.49€,Best Sellers
4,SOME BY MI - AHA-BHA-PHA 30days Miracle Acne C...,Product Type: Cleansing Water\n\n\nSkin Concer...,[],15.09€,Best Sellers
...,...,...,...,...,...
1082,ON THE BODY - The Natural Olive Cleansing Foam...,no info,no info,no info,no
1083,COW soap - Skinlife Medicated Acne Care Facial...,no info,no info,no info,no
1084,ACWELL - No5.5 Bubble-free pH Balancing Cleans...,no info,no info,no info,no
1085,3W Clinic - Placenta Clear Foam Cleansing -180ml,no info,no info,no info,no


In [29]:
cleansers = cleansers_df.to_csv('../data_/cleansers.csv')

In [31]:
cleansers_df[cleansers_df['ingredients'] == 'no info']

,name,details,ingredients,price,best-seller
5,Haruharu WONDER - Black Rice Moisture Deep Cle...,no info,no info,no info,no
8,BE PLAIN - Greenful Ph-Balanced Cleansing Foam...,no info,no info,no info,no
10,SOME BY MI - AHA-BHA-PHA 30 Days Miracle Clean...,no info,no info,no info,no
11,iUNIK - Centella Bubble Cleansing Foam - 150ml,no info,no info,no info,no
12,Farm Stay - 76 Green Tea Seed Premium Moisture...,no info,no info,no info,no
...,...,...,...,...,...
1082,ON THE BODY - The Natural Olive Cleansing Foam...,no info,no info,no info,no
1083,COW soap - Skinlife Medicated Acne Care Facial...,no info,no info,no info,no
1084,ACWELL - No5.5 Bubble-free pH Balancing Cleans...,no info,no info,no info,no
1085,3W Clinic - Placenta Clear Foam Cleansing -180ml,no info,no info,no info,no


In [3]:
df1 = pd.read_csv('../stylevana_data/cleansers.csv')
df2 = pd.read_csv('../data_/cleansers.csv')